In [1]:
import pandas as pd
import unidecode
from string import punctuation
from nltk import RSLPStemmer
from nltk.tokenize import WordPunctTokenizer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans

In [6]:
from nltk import download as nltkd
nltkd('stopwords')
nltkd('rslp')

[nltk_data] Downloading package stopwords to /home/nanny/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package rslp to /home/nanny/nltk_data...
[nltk_data]   Unzipping stemmers/rslp.zip.


True

In [7]:
token_pontuacao = WordPunctTokenizer()
palavras_irrelevantes = stopwords.words("portuguese")
stemmer = RSLPStemmer()
stopwords_sem_acento = [unidecode.unidecode(texto) for texto in ([ponto for ponto in punctuation] + palavras_irrelevantes)]

In [8]:
def tratamento(coluna_texto):
    frase_processada = list()
    for opiniao in coluna_texto:
        nova_frase = list()
        palavras_texto = token_pontuacao.tokenize(
            unidecode.unidecode(str(opiniao).lower()))
        for palavra in palavras_texto:
            if palavra not in stopwords_sem_acento:
                nova_frase.append(stemmer.stem(palavra))
        frase_processada.append(' '.join(nova_frase))
    return frase_processada

In [18]:
df= pd.read_excel("evt_202212201043.xlsx", engine='openpyxl')

In [19]:
df.sample(3)

,EVENTO,DSC_EVT
1127,2022/253561,Tais Login por cpf da funcionária - CNPJ 1090...
604,2022/247094,Mensageria: Protocolo: 2022/13763 - Duvidas so...
1272,2022/255122,Atendimento por e-mail\nSr. Alef Alves\nFwd: B...


In [20]:
df.rename(columns={"EVENTO":"id","DSC_EVT":"texto"}, inplace=True)
df.sample(3)

,id,texto
128,2022/240012,Info sobre confirmação de pedido de compra. CN...
1938,2022/263739,Atendimento via Teams - Sr. Leandro Santana de...
154,2022/240421,Fornecedor informou que não conseguiu realizar...


In [21]:
df['pre'] = tratamento(df['texto'])

In [33]:
vetor_tfidf = TfidfVectorizer(lowercase=False, ngram_range=(1, 2)).fit_transform(df['pre'])

In [32]:
df['cat'] = KMeans(n_clusters=4, verbose=False).fit(vetor_tfidf).predict(vetor_tfidf)

In [35]:
df[df['cat']==0][['id','texto','cat']].sample(20)

,id,texto,cat
1259,2022/255000,Atendimento Telefônico\nSra. Paula - (11) 972...,0
584,2022/246923,DÚVIDA - PARCELA/COMPETÊNCIA PORTAL DO FORNECE...,0
1508,2022/258063,Dúvidas sobre o recebimento de e-mail da Gefor...,0
1608,2022/258855,Mensageria - Protocolo: 2022/51019\nIndisponib...,0
809,2022/250517,Atendimento por e-mail\nSolicitaram orientação...,0
1552,2022/258392,upload de nota fiscal/SICAF,0
889,2022/251058,Atendimento por e-mail\nBB/ GEFOR 34 - Balanço...,0
1272,2022/255122,Atendimento por e-mail\nSr. Alef Alves\nFwd: B...,0
217,2022/241224,Atendimento Telefônico - Sr. Luiz - (91) 98331...,0
838,2022/250647,balanço Patrimonial - questiona a qual contrat...,0


In [28]:
df[df['cat']==1][['id','texto','cat']].sample(10)

,id,texto,cat
1604,2022/258766,NF eletrônica enviada via e-mail,1
1212,2022/254586,assinatura contrato 202274213787 - signatário ...,1
1127,2022/253561,Tais Login por cpf da funcionária - CNPJ 1090...,1
2023,2022/265105,Vinicius - CNPJ 01.451.373/0001-41 Contrato...,1
594,2022/247038,Fluxo cancelado - cadastrado incorretamente,1
1433,2022/257079,Atendimento por e-mail\nRe: ACEITE - Contrato ...,1
520,2022/245638,RES: MANUAL DE INCLUSÃO DE NOTAS E PLANILHA DE...,1
75,2022/239253,Auxilio cadastro e envio NF. CNPJ. .: 30.092.8...,1
1354,2022/256238,Fernanda CNPJ 44652512000108 problema no doc p...,1
107,2022/239754,Envio NF. CNPJ. .: 05.607.266/0001-10 \n,1


In [29]:
df[df['cat']==2][['id','texto','cat']].sample(10)

,id,texto,cat
924,2022/251608,Balanço Patrimonial,2
878,2022/250981,balanço patrimonial/ orientações envio,2
844,2022/250653,Balanço Patrimonial,2
852,2022/250672,Atendimento por e-mail\nENC: BB/ GEFOR 34 - Ba...,2
848,2022/250665,Atendimento por e-mail\nFwd: Fwd: BB/ GEFOR 34...,2
837,2022/250646,Balanço Patrimonial,2
853,2022/250673,Atendimento telefônico realizado em 12.05.2022...,2
1081,2022/252970,Balanço Patrimonial - upload da NF - Marcos - ...,2
880,2022/250986,confirmação envio balanço Patrimonial,2
824,2022/250563,Atendimento Telefônico\nSr. Fabricio - (99) 98...,2


In [30]:
df[df['cat']==3][['id','texto','cat']].sample(10)

,id,texto,cat
1336,2022/256068,pagamento NF contrato 201974218057,3
1313,2022/255865,emissão e pagamento de NF,3
801,2022/250328,emissão de NF,3
778,2022/250053,erro upload NF,3
1243,2022/254852,upload de NF,3
526,2022/245728,envio de NF - contrato 201785580039,3
1328,2022/255964,Pagamento de NF contrato 202074217676,3
407,2022/243820,envio de nf,3
411,2022/243832,envio de NF/ instabilidade no Portal,3
706,2022/248854,pagamento de NF,3


In [34]:
df['cat'].value_counts()

2    1923
0     307
1      89
3      64
Name: cat, dtype: int64